## Airport Capacity Simulation Exercise

I will attempt to simulate a simplified airport security system at a busy airport. 

These are the assumptions i will be making in this simulation:

1. Passengers arrive according to a Poisson distribution with λ1 = 50 per minute (i.e., mean interarrival rate $\lambda_1$ = 0.02 minutes) to the ID/boarding-pass check queue, where there are several servers who each have exponential service time with mean rate $\lambda_2$ = 0.75 minutes. [Hint: model them as one block that has more than one resource.] 

2. After that, the passengers are assigned to the shortest of the several personal-check queues, where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1 minute).


I will build a simulation of the system, and then vary the number of ID/boarding-pass checkers and personal-check queues to determine how many are needed to keep average wait times below 15 minutes.

#### Main Approach

I will summarise my main approach to this simulation before diving into the code.

<br> **Objective:** The goal of this simulation is to look for the optimal number of resources (id checkers and scanners) required to keep average wait times below 15 minutes. </br>
<br>**Steps:**</br>
1. Import necessary libraries and declare global variables
2. Create Airport class and two functions for checking passes and passengers entering the personal scan
3. Define `security_clearance` function to move passenger through the airport and record timestamps as passenger moves through the airport
4. Define `passengers_arrive ` function to generate passengers arriving at the airport for security clearance. Calls the `security_clearance` function.
5. Define function to calculate average wait time for simulation
6. Instantiate Simpy environment and run simulation for 10 hours with a set number of id checkers and scanners.
7. Obtain optimal number of resources required to keep wait times under 15 minutes.

Step 1: Import necessary libraries and declare global variables

In [1]:
import random
import simpy
import statistics
import numpy
import pandas as pd
pd.set_option("display.max_rows", 50)


In [2]:
wait_times = []
mean_checker_time = 0.75
scan_time_range = [0.5, 1]


Step 2: Create Airport class and two functions for checking passes and passengers entering the personal scan

In [3]:
# Creates the Airport class and the two functions:
# (1) pass checking (2) personal scanning

class Airport(object):

    wash_times = []

    def __init__(self, env, num_idpass_checkers, num_personal_scanners):
        self.env = env
        self.checker = simpy.Resource(env, num_idpass_checkers)
        self.scanner = simpy.Resource(env, num_personal_scanners)

    def check_pass(self, passenger, mean_checker_time):
        yield self.env.timeout(numpy.random.exponential(mean_checker_time))

    def personal_scan(self, passenger, scan_time_range):
        yield self.env.timeout(random.uniform(scan_time_range[0], scan_time_range[1]))


Step 3: Define `security_clearance` function to move passenger through the airport and record timestamps as passenger moves through the airport


In [4]:
# Create function to move passenger through the airport security system

def security_clearance(env, passenger, airport):
        
    arrival_time = env.now
    # print(f"Passenger number {passenger + 1} entering security clearance at {arrival_time}")

    with airport.checker.request() as request:
        # request for a checker
        yield request
        # check the passenger's ID
        yield env.process(airport.check_pass(passenger, mean_checker_time))
        # print(f"Passenger {passenger} passes airport checker at {env.now}.")
    with airport.scanner.request() as request:
        # request for a scanner
        yield request
        # scan the passenger's luggage
        yield env.process(airport.personal_scan(passenger, scan_time_range))
        # print(f"Passenger {passenger} passes scanner at {env.now}.")

    wait_times.append(env.now - arrival_time)


Step 4: Define `passengers_arrive ` function to generate passengers arriving at the airport for security clearance. Calls the `security_clearance` function.


In [5]:
# Generate a passenger after arrival-rate of 50 passengers/minute 
# and increment number of passengers

def passengers_arrive(env, airport):
    # airport = Airport(env, num_idpass_checkers, num_personal_scanners)
    passenger = 0
    
    while True:
        # Wait 1/50 minutes before another passenger
        yield env.timeout(numpy.random.exponential(0.02)) 

        passenger += 1
        env.process(security_clearance(env, passenger, airport))
        # print(f"Passenger number {passenger} exiting at {env.now}")
        


Step 5: Define function to calculate average wait time for simulation


In [6]:
# Calculate wait time and present data

def average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    # minutes, frac_minutes = divmod(average_wait, 1)
    # seconds = frac_minutes * 60
    return round(average_wait, 2)

Step 6: Instantiate Simpy environment and run simulation for 10 hours with a set number of id checkers and scanners.


In [7]:
# Create a Simpy Environment
random.seed(123)
env = simpy.Environment()

# Create an airport 
# Start simulation
id_checker_range = range(40,50)
scanner_range = range(30,50)
simulation_duration = 10 * 60

under_50 = 50
target_time = 15

results_df = pd.DataFrame(columns=["id_checkers", "scanners", "time"])

for id_checker in id_checker_range:
    for scanner in scanner_range:
        env = simpy.Environment()
        airport = Airport(env, id_checker, scanner)
        env.process(passengers_arrive(env, airport))
        env.run(until=simulation_duration)

        # View results
        time = average_wait_time(wait_times)
        print(f"Average time taken {time} with {id_checker} id checkers and {scanner} scanners.")
        new_df = pd.DataFrame({"id_checkers" : [id_checker], "scanners" : [scanner], "time" : [time]})
        results_df = pd.concat([results_df, new_df], ignore_index=True)

        if time <= target_time:
            print(f"Time achieved: {time}")
            print(f"The optimal number of resources to obtain a time under 15 minutes are {id_checker} id_checkers and {scanner} scanners.")
            break
        else:
            continue
    
    if time <= target_time:
        break
    else:
        continue

        


Average time taken 58.87 with 40 id checkers and 30 scanners.
Average time taken 55.38 with 40 id checkers and 31 scanners.
Average time taken 52.32 with 40 id checkers and 32 scanners.
Average time taken 49.01 with 40 id checkers and 33 scanners.
Average time taken 44.83 with 40 id checkers and 34 scanners.
Average time taken 41.22 with 40 id checkers and 35 scanners.
Average time taken 36.67 with 40 id checkers and 36 scanners.
Average time taken 32.49 with 40 id checkers and 37 scanners.
Average time taken 28.78 with 40 id checkers and 38 scanners.
Average time taken 25.84 with 40 id checkers and 39 scanners.
Average time taken 23.45 with 40 id checkers and 40 scanners.
Average time taken 21.52 with 40 id checkers and 41 scanners.
Average time taken 19.91 with 40 id checkers and 42 scanners.
Average time taken 18.53 with 40 id checkers and 43 scanners.
Average time taken 17.34 with 40 id checkers and 44 scanners.
Average time taken 16.31 with 40 id checkers and 45 scanners.
Average 

I also take this opportunity to view the progression of wait time reduction as we increase the number of resources to process passenger security clearance.

In [8]:
results_df

,id_checkers,scanners,time
0,40,30,58.87
1,40,31,55.38
2,40,32,52.32
3,40,33,49.01
4,40,34,44.83
5,40,35,41.22
6,40,36,36.67
7,40,37,32.49
8,40,38,28.78
9,40,39,25.84


Step 7: Obtain optimal number of resources required to keep wait times under 15 minutes.

In conclusion, we see that with a busier airport of 50 passengers per minute, we will require 40 id checking servers and 47 personal scanners to keep wait times under 15 minutes.